### LCZ

In [1]:
# !wget https://zenodo.org/records/8419340/files/lcz_filter_v3.tif?download=1

--2025-01-15 11:41:45--  https://zenodo.org/records/8419340/files/lcz_filter_v3.tif?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384822872 (1,3G) [image/tiff]
Saving to: ‘lcz_filter_v3.tif?download=1’

lcz_filter_v3.tif?d 100%[===================>]   1,29G  5,49MB/s    in 3m 42s  

2025-01-15 11:45:27 (5,95 MB/s) - ‘lcz_filter_v3.tif?download=1’ saved [1384822872/1384822872]



In [8]:
# !mv lcz_filter_v3.tif /data/uscuni-ulce/validation/

In [2]:
import rioxarray
import geopandas as gpd
import numpy as np

furl = "/data/uscuni-ulce/validation/lcz_filter_v3.tif"

In [3]:
lczs = rioxarray.open_rasterio(furl).squeeze()

In [4]:
lczs

<xarray.DataArray (y: 155995, x: 389620)> Size: 61GB
[60778771900 values with dtype=uint8]
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3MB -170.0 -170.0 -170.0 ... 180.0 180.0 180.0
  * y            (y) float64 1MB 80.04 80.04 80.04 ... -60.09 -60.09 -60.09
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0

In [5]:
region_id = 5883

In [6]:
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/{region_id}_clusters.pq')

In [7]:
# setup geometry list to burn into the raster
buildings['num_index'] = np.arange(1, buildings.shape[0] + 1)
buildings['geometry'] = buildings.representative_point()

building_utm = buildings.to_crs(lczs.rio.crs)
minx, miny, maxx, maxy = building_utm.total_bounds

geom = building_utm.reset_index()[['geometry', 'num_index']].values.tolist()

In [23]:
# read only the region part of the raster data
region_lczs = rioxarray.open_rasterio(
    filename=furl, 
    chunks='auto', 
    parse_coordinates=True,
    masked=True
).rio.clip_box(minx, miny, maxx, maxy)

In [10]:
%%time
from rasterio import features
fields_rasterized = features.rasterize(geom, out_shape=(region_lczs.shape[1], region_lczs.shape[2]), transform=region_lczs.rio.transform())

CPU times: user 423 ms, sys: 6.1 ms, total: 430 ms
Wall time: 426 ms


In [11]:
np.unique(fields_rasterized).shape

(8511,)

In [13]:
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr

import xvec

In [20]:
lczs

<xarray.DataArray (y: 155995, x: 389620)> Size: 61GB
[60778771900 values with dtype=uint8]
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3MB -170.0 -170.0 -170.0 ... 180.0 180.0 180.0
  * y            (y) float64 1MB 80.04 80.04 80.04 ... -60.09 -60.09 -60.09
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0

In [16]:
building_utm

,index,currentUse,buildingNature,heightAboveGround,iid,geometry,label,morph,final,regional,final_without_noise,num_index
0,0,None,None,NaN,12613800,POINT (7.8251 48.01675),179,5883_0_259,5,5883_34,5,1
1,1,None,None,NaN,12614053,POINT (7.85089 48.0671),502,5883_1_0,9,5883_82,9,2
2,2,None,None,NaN,12614101,POINT (7.8279 47.9541),15,5883_0_110,30,5883_85,30,3
3,3,None,None,NaN,12614108,POINT (7.86118 47.98688),93,5883_0_181,0,5883_32,0,4
4,4,None,None,NaN,12614231,POINT (7.80564 48.01521),204,5883_0_281,9,5883_67,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...
70888,70891,None,None,NaN,19047465,POINT (7.8622 48.01139),327,5883_0_7,0,5883_41,0,70889
70889,70892,None,None,NaN,19047548,POINT (7.7969 47.97944),122,5883_0_207,9,5883_66,9,70890
70890,70893,None,None,NaN,19047781,POINT (7.84791 48.06636),504,5883_1_10,30,5883_85,30,70891
70891,70894,None,None,NaN,19047819,POINT (7.80405 48.0741),443,5883_15_6,30,5883_81,30,70892


In [25]:
aggregated_iterative = region_lczs.xvec.extract_points(
    building_utm.geometry,
    x_coords="x",
    y_coords="y",
    # method="iterate",
    # stats='max',
    # n_jobs=-1,
)
aggregated_iterative

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker sto

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x7c850c4b1dc0>' as a data type

In [27]:
aggregated_iterative

NameError: name 'aggregated_iterative' is not defined

### Urban atlas

In [1]:
!wget https://copernicus-fme.eea.europa.eu/clmsdatadownload/results/141771.zip

--2025-01-16 10:24:37--  https://copernicus-fme.eea.europa.eu/clmsdatadownload/results/141771.zip
Resolving copernicus-fme.eea.europa.eu (copernicus-fme.eea.europa.eu)... 98.71.17.92
Connecting to copernicus-fme.eea.europa.eu (copernicus-fme.eea.europa.eu)|98.71.17.92|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 340073497 (324M) [application/zip]
Saving to: ‘141771.zip’

141771.zip          100%[===================>] 324,32M  6,93MB/s    in 59s     

2025-01-16 10:25:36 (5,53 MB/s) - ‘141771.zip’ saved [340073497/340073497]



In [1]:
!mv 141771.zip /data/uscuni-ulce/validation/